In [ ]:
!pip install opencv-python numpy matplotlib tensorflow


In [ ]:
# --- 0) Imports
import os, cv2, numpy as np, tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt

In [ ]:
# --- 1) Config
MODEL_PATH = "models/maiz_efficientnetb3.h5"   # cambia a tu ruta
CLASSES = ["common_rust", "gray_leaf_spot", "northern_leaf_blight", "healthy"]
THRESH_OTHER = 0.60     # umbral para decidir "otra" (ajustar con validación)
CLAHE_CLIP = 2.0
CLAHE_TILE = (8, 8)

In [ ]:
# --- 2) Cargar modelo
model = keras.models.load_model(MODEL_PATH, compile=False)
H, W = model.input_shape[1], model.input_shape[2]

In [ ]:
# --- 3) Preprocesado del paper: RGB→Lab, CLAHE(L), mediana, back→RGB, resize y escalar
def preprocess_rgb(img_rgb, size=(H, W)):
    lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)
    L, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=CLAHE_CLIP, tileGridSize=CLAHE_TILE)
    Lc = clahe.apply(L)
    lab_c = cv2.merge([Lc, a, b])
    rgb_c = cv2.cvtColor(lab_c, cv2.COLOR_LAB2RGB)
    rgb_c = cv2.medianBlur(rgb_c, ksize=3)   # aproximación a AMF
    rgb_r = cv2.resize(rgb_c, size, interpolation=cv2.INTER_AREA)
    x = rgb_r.astype(np.float32) / 255.0
    return x

def load_image_for_infer(path):
    bgr = cv2.imread(path, cv2.IMREAD_COLOR)
    if bgr is None:
        raise ValueError(f"No se pudo leer: {path}")
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    x = preprocess_rgb(rgb)
    return rgb, np.expand_dims(x, 0)

In [ ]:
# --- 4) Inferencia + “otra”
def predict_image(path):
    rgb_orig, x = load_image_for_infer(path)
    probs = model.predict(x, verbose=0)[0]
    k = int(np.argmax(probs))
    p = float(probs[k])
    label = CLASSES[k]
    is_other = p < THRESH_OTHER
    return {"orig": rgb_orig, "probs": probs, "label": label, "pmax": p, "is_other": is_other}

In [ ]:
# --- 5) Grad-CAM para visualizar y estimar % de área afectada (aprox)
def last_conv_layer(m):
    # intenta detectar la última capa conv2d si no sabés el nombre
    for layer in reversed(m.layers):
        if isinstance(layer, keras.layers.Conv2D):
            return layer.name
    raise ValueError("No se encontró Conv2D")

def grad_cam(m, img_array, class_index=None, layer_name=None):
    if layer_name is None:
        layer_name = last_conv_layer(m)
    conv_layer = m.get_layer(layer_name)
    grad_model = keras.models.Model([m.inputs], [conv_layer.output, m.outputs])
    with tf.GradientTape() as tape:
        conv_out, preds = grad_model(img_array, training=False)
        if class_index is None:
            class_index = int(tf.argmax(preds[0]))
        class_channel = preds[:, class_index]
    grads = tape.gradient(class_channel, conv_out)
    weights = tf.reduce_mean(grads, axis=(1, 2), keepdims=True)
    cam = tf.nn.relu(tf.reduce_sum(weights * conv_out, axis=-1))[0]
    cam = cam.numpy()
    cam -= cam.min()
    cam /= (cam.max() + 1e-8)
    cam = cv2.resize(cam, (img_array.shape[2], img_array.shape[1]))
    return cam

def lesion_percent_from_cam(cam, q=0.7):
    thr = np.quantile(cam, q)
    mask = (cam >= thr).astype(np.uint8)
    return 100.0 * mask.mean(), mask


In [ ]:
# --- 6) Ejecutar sobre una foto tuya
# path_foto = "mis_fotos/hoja_maiz.jpg"
# out = predict_image(path_foto)
# cam = grad_cam(model, out_img := np.expand_dims(preprocess_rgb(out["orig"]), 0))
# perc, mask = lesion_percent_from_cam(cam, q=0.7)

In [ ]:
# --- 7) Mostrar resultados
def show_result(path_foto):
    out = predict_image(path_foto)
    x = np.expand_dims(preprocess_rgb(out["orig"]), 0)
    cam = grad_cam(model, x)
    perc, mask = lesion_percent_from_cam(cam, q=0.7)

    title = "otra" if out["is_other"] else out["label"]
    print(f"pred: {title}  pmax={out['pmax']:.3f}")
    if not out["is_other"]:
        print(f"% área afectada (aprox, CAM>=p{70}): {perc:.1f}%")

    # overlays
    heat = (plt.cm.jet(cam)[:, :, :3] * 255).astype(np.uint8)
    overlay = cv2.addWeighted(out["orig"], 0.6, heat, 0.4, 0)
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1); plt.imshow(out["orig"]); plt.axis("off"); plt.title("original")
    plt.subplot(1,3,2); plt.imshow(overlay);    plt.axis("off"); plt.title("Grad-CAM")
    plt.subplot(1,3,3); plt.imshow(mask, cmap="gray"); plt.axis("off"); plt.title("lesión binaria")
    plt.show()




In [ ]:
# Ejemplo:
# show_result(path_foto)